In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
from sklearn.tree import DecisionTreeClassifier

In [10]:
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']} 
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\swath\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
7 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\swath\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\swath\anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\swath\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\swath\anaconda3\Lib\site-packages\s

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [11]:
re=grid.cv_results_
print(re)
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

{'mean_fit_time': array([0.00131721, 0.00107017, 0.00310273, 0.00161333, 0.00242734,
       0.00272121, 0.00121117, 0.00133939, 0.00241265, 0.00096169,
       0.00167499, 0.0004014 ]), 'std_fit_time': array([0.00263443, 0.00159816, 0.00064459, 0.00136963, 0.00128061,
       0.00140807, 0.0016074 , 0.00071905, 0.00152634, 0.00117782,
       0.00157916, 0.0008028 ]), 'mean_score_time': array([0.        , 0.        , 0.00642977, 0.00650058, 0.00712242,
       0.00557895, 0.        , 0.        , 0.00234451, 0.00659375,
       0.00427132, 0.0097322 ]), 'std_score_time': array([0.        , 0.        , 0.00206329, 0.00327296, 0.0027421 ,
       0.00080146, 0.        , 0.        , 0.00277432, 0.00348356,
       0.00141227, 0.00373796]), 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'gini', 'gini',
                   'entropy', 'entropy', 'entropy', 'entropy', 'entropy',
                   'entropy'],
             mask=[False, False, False, False, False, False, False, Fa

In [12]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'log2', 'splitter': 'best'}: 0.8643575957337304


In [13]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[78  7]
 [11 38]]


In [14]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.88      0.92      0.90        85
           1       0.84      0.78      0.81        49

    accuracy                           0.87       134
   macro avg       0.86      0.85      0.85       134
weighted avg       0.86      0.87      0.86       134



In [15]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8465786314525809

In [16]:
table=pd.DataFrame.from_dict(re)

In [17]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001317,0.002634,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.001070,0.001598,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.003103,0.000645,0.006430,0.002063,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.829615,0.815037,0.795256,0.867097,0.844370,0.830275,0.024550,7
3,0.001613,0.001370,0.006501,0.003273,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.869532,0.813511,0.815056,0.887907,0.868632,0.850928,0.030705,5
4,0.002427,0.001281,0.007122,0.002742,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.829615,0.906252,0.833323,0.868632,0.905069,0.868578,0.033202,1
5,0.002721,0.001408,0.005579,0.000801,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.867478,0.870047,0.777291,0.832483,0.847020,0.838864,0.033733,6
6,0.001211,0.001607,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.001339,0.000719,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.002413,0.001526,0.002345,0.002774,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.847141,0.907177,0.775815,0.925272,0.847020,0.860485,0.052737,2
9,0.000962,0.001178,0.006594,0.003484,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.791752,0.773585,0.777291,0.831098,0.901744,0.815094,0.047878,8


In [41]:
age = float(input("Enter Age: "))
estimated_salary = float(input("Enter Estimated Salary: "))
gender_male = int(input("Enter Gender (Male=1, Female=0): "))

Enter Age:  19
Enter Estimated Salary:  19000
Enter Gender (Male=1, Female=0):  1


In [43]:
Future_Prediction=grid.predict([[age,estimated_salary,gender_male]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
